<a href="https://colab.research.google.com/github/AndromedaOMA/Advanced_Analytics_with_Apache_Spark---E.On_Software_Development/blob/main/Laboratory_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pregătire mediu de lucru


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#Check this site for the latest download link https://dlcdn.apache.org/spark/
!wget -q https://dlcdn.apache.org/spark/spark-3.4.4/spark-3.4.4-bin-hadoop3.tgz
!tar xf spark-3.4.4-bin-hadoop3.tgz
!pip install -q findspark
!pip install pyspark
!pip install py4j
import os
import sys
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
# os.environ["SPARK_HOME"] = "/content/spark-3.4.4-bin-hadoop3"
import findspark
findspark.init()
findspark.find()
import pyspark

from pyspark.sql import DataFrame, SparkSession
from typing import List
import pyspark.sql.types as T
import pyspark.sql.functions as f
spark= SparkSession.builder.getOrCreate()
spark

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,788 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,243 kB]
Get:13 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Package

# Înlănțuirea metodelor

## Java Builder

În programarea orientată obiect (OOP), design pattern-urile sunt șabloane pentru a rezolva probleme comune de
design a software-ului. Unul dintre cele mai cunoscute design pattern-uri este Builder, provenit din limbajul Java.

Implementarea șablonului și în Python:

In [ ]:
class CarBuilder:
  # Inițializarea builder-ului cu valori implicite potrivite.
  def __init__(self):
    self.car = {'wheels': 4, 'color': 'Silver'}
  # Funcțiile de setare a opțiunilor. Șablonul de builder se remarcă prin returnarea obiectul de self la metodele sale de setare. Asta permite înlănțuirea metodelor:
  def set_wheels(self, number):
    self.car['wheels'] = number
    return self
  def set_color(self, color):
    self.car['color'] = color
    return self
  # Funcțiile de finalizare, care returnează rezultatul
  def build(self):
    return self.car

## Java Builder – Utilizare in PySpark

Având la bază limbajul Scala, bazat pe Java, și fiind și un Framework bazat pe construcții de transformări, Spark, și
respectiv PySpark, se folosesc de acest șablonul de programare Builder în API-ul pe care îl oferă.

Am întâlnit până acum aceste construcții:

In [3]:
from pyspark.sql import types as T
data_schema = T.StructType([
T.StructField('nume', T.StringType(), False),
T.StructField('varsta', T.IntegerType(), False),
T.StructField('ocupatie', T.StringType(), False),
T.StructField('vechime', T.IntegerType(), False),
T.StructField('inactiv', T.BooleanType(), True),
T.StructField('zona', T.StringType(), True),
T.StructField('extra', T.ArrayType(T.StringType()), True)
])

spark = SparkSession.builder.master('local[*]').config('spark.driver.memory', '3g').getOrCreate()
#       ---------------------======================================================______________
# Inițializarea builder-ului|       Apelarea funcțiilor de schimbare a           | Apelarea funcției de finalizare
# cu valori implicite       |            opțiunilor implicite.                   | care returnează obiectul
# potrivite alese de        |                                                    | constuit Spark/Data Frame
#         -----------===========================================================_______________________________
data_df = spark.read.format('csv').option('header', 'true').schema(data_schema).load('/path/to/folder/or/file')

# Comenzi de transformare

*Ce nu trebuie, nu facem!*

## Datele de test (createDataFrame)

Pentru a testa cod de Spark, în general, se folosesc liste mici cu date de test la crearea unui Data Frame, în loc să
citim datele. Când testam aplicații de Spark, putem folosi această metodă pentru scrierea de test.

➢ Funcția **createDataFrame** ne permite să specificăm doar numele coloanelor, fără a fi nevoie să
specificăm și tipul lor. Spark va trece prin data și va încerca să deducă automat tipul de date.

In [5]:
data = [
['Vali', 23, 'Programator', 4, None, 'A' , ['3D Printer', 'XBOX']],
['Vlad', 34, 'Instalator', 11, None, 'B', ['EV']],
['Bea', 29, 'Reporter', 7, True, 'B', None]
]
data_df = spark.createDataFrame(data, schema=['nume', 'varsta', 'ocupatie', 'vechime', 'inactiv', 'zona', 'extra'])

## Proiectarea Datelor - Select (select)

Data Frame-urile oferă și metode pentru proiectarea datelor, pe lângă cele de scriere, colectare și afișare, similare
cu interogările din bazele de date SQL, în același stil de apelare prin înlănțuire.

➢ Operația de Selecție – Păstrarea doar anumite coloane

In [6]:
new_data_df = data_df.select('nume', 'varsta', 'extra')

❖ Un nou obiect de tip Data Frame este returnat care are doar coloanele nume, varsta și extra. Data Frame-ul
de la care a pornit proiecția va rămâne neschimbat.

In [7]:
data_df.show()

+----+------+-----------+-------+-------+----+------------------+
|nume|varsta|   ocupatie|vechime|inactiv|zona|             extra|
+----+------+-----------+-------+-------+----+------------------+
|Vali|    23|Programator|      4|   NULL|   A|[3D Printer, XBOX]|
|Vlad|    34| Instalator|     11|   NULL|   B|              [EV]|
| Bea|    29|   Reporter|      7|   true|   B|              NULL|
+----+------+-----------+-------+-------+----+------------------+



In [8]:
new_data_df.show()

+----+------+------------------+
|nume|varsta|             extra|
+----+------+------------------+
|Vali|    23|[3D Printer, XBOX]|
|Vlad|    34|              [EV]|
| Bea|    29|              NULL|
+----+------+------------------+



## Proiectarea Datelor - Ștergere (drop)

De multe ori se întâmplă să lucrăm cu date care conțin foarte multe coloane, și devine inpractic să le selectăm. În
loc de selecția coloanelor pe care dorim să le păstrăm, putem scoate coloanele de care nu avem nevoie.

➢ Operația de Ștergere – Ștergere a coloanelor

In [9]:
new_data_df = data_df.drop('varsta', 'zona', 'extra')

❖ Un nou obiect de tip Data Frame este returnat care nu mai are coloanele varsta, concediu și
extra. Data Frame-ul de la care a pornit proiecția va rămâne neschimbat.

In [10]:
data_df.show()

+----+------+-----------+-------+-------+----+------------------+
|nume|varsta|   ocupatie|vechime|inactiv|zona|             extra|
+----+------+-----------+-------+-------+----+------------------+
|Vali|    23|Programator|      4|   NULL|   A|[3D Printer, XBOX]|
|Vlad|    34| Instalator|     11|   NULL|   B|              [EV]|
| Bea|    29|   Reporter|      7|   true|   B|              NULL|
+----+------+-----------+-------+-------+----+------------------+



In [11]:
new_data_df.show()

+----+-----------+-------+-------+
|nume|   ocupatie|vechime|inactiv|
+----+-----------+-------+-------+
|Vali|Programator|      4|   NULL|
|Vlad| Instalator|     11|   NULL|
| Bea|   Reporter|      7|   true|
+----+-----------+-------+-------+



## Proiectarea Datelor – Redenumirea Coloanelor (withColumnRenamed)

Numele coloanele din date adesea nu se potrivesc cu metodologia noastă de a numi coloanele, de exemplu
conțin caractere neobișnuite sau sunt în altă limbă. Putem să redenumim aceste coloane pentru a le standardiza.

➢ Operația de Redenumire – Redenumire a coloanelor

In [12]:
new_data_df = data_df.withColumnRenamed('ocupatie', 'job')

❖ Un nou obiect de tip Data Frame este returnat care nu mai are coloana post, dar are o nouă coloană job cu
aceleași valori. Data Frame-ul de la care a pornit proiecția va rămâne neschimbat.

In [13]:
data_df.printSchema()

root
 |-- nume: string (nullable = true)
 |-- varsta: long (nullable = true)
 |-- ocupatie: string (nullable = true)
 |-- vechime: long (nullable = true)
 |-- inactiv: boolean (nullable = true)
 |-- zona: string (nullable = true)
 |-- extra: array (nullable = true)
 |    |-- element: string (containsNull = true)



In [14]:
new_data_df.printSchema()

root
 |-- nume: string (nullable = true)
 |-- varsta: long (nullable = true)
 |-- job: string (nullable = true)
 |-- vechime: long (nullable = true)
 |-- inactiv: boolean (nullable = true)
 |-- zona: string (nullable = true)
 |-- extra: array (nullable = true)
 |    |-- element: string (containsNull = true)

